In [26]:
# 라이브러리 import
import pandas as pd
import numpy as np
import statistics as st

# 경고 메세지 무시
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [17]:
prd = pd.read_csv('product_processed.csv')
evt = pd.read_csv('event_processed.csv')
df = pd.merge(evt,prd, how = 'inner', on = 'item_no')
del prd
del evt

In [27]:
# 주차 컬럼 추가
df['w'] = pd.to_datetime(df.event_timestamp).dt.week

df.loc[(df.event_timestamp > '2021-06-03') & (df.event_timestamp <= '2021-06-10'),'w'] = '2021-06-03'
df.loc[(df.event_timestamp > '2021-06-10') & (df.event_timestamp <= '2021-06-17'),'w'] = '2021-06-10'
df.loc[(df.event_timestamp > '2021-06-17') & (df.event_timestamp <= '2021-06-24'),'w'] = '2021-06-17'
df.loc[(df.event_timestamp > '2021-06-24') & (df.event_timestamp <= '2021-07-01'),'w'] = '2021-06-24'
df.loc[(df.event_timestamp > '2021-07-01') & (df.event_timestamp <= '2021-07-08'),'w'] = '2021-07-01'
df.loc[(df.event_timestamp > '2021-07-08') & (df.event_timestamp <= '2021-07-15'),'w'] = '2021-07-08'
df.loc[(df.event_timestamp > '2021-07-15') & (df.event_timestamp <= '2021-07-22'),'w'] = '2021-07-15'
df.loc[(df.event_timestamp > '2021-07-22') & (df.event_timestamp <= '2021-07-29'),'w'] = '2021-07-22'
df.loc[(df.event_timestamp > '2021-07-29'),'w'] = '2021-07-29'

#### 의류, 잡화 상승률 및 기하 평균 계산

In [23]:
# 카테고리 중분류 및 주차별 매출액 피벗테이블 생성
tmp = pd.pivot_table(data = df[(df.event_name == 'purchase_success')&((df.category1_name == '의류')|(df.category1_name == '잡화'))], index = 'category2_name', columns = 'w',values = 'price', aggfunc = 'sum').reset_index()

In [24]:
# 주 단위 매출 상승률 계산
tmp['2021-06-10_pct'] = (tmp['2021-06-10'] - tmp['2021-06-03'])/tmp['2021-06-03']
tmp['2021-06-17_pct'] = (tmp['2021-06-17'] - tmp['2021-06-10'])/tmp['2021-06-10']
tmp['2021-06-24_pct'] = (tmp['2021-06-24'] - tmp['2021-06-17'])/tmp['2021-06-17']
tmp['2021-07-01_pct'] = (tmp['2021-07-01'] - tmp['2021-06-24'])/tmp['2021-06-24']
tmp['2021-07-08_pct'] = (tmp['2021-07-08'] - tmp['2021-07-01'])/tmp['2021-07-01']
tmp['2021-07-15_pct'] = (tmp['2021-07-15'] - tmp['2021-07-08'])/tmp['2021-07-08']
tmp['2021-07-22_pct'] = (tmp['2021-07-22'] - tmp['2021-07-15'])/tmp['2021-07-15']
tmp['2021-07-29_pct'] = (tmp['2021-07-29'] - tmp['2021-07-22'])/tmp['2021-07-22']

In [25]:
# 주 단위 평균 매출 상승률 계산(기하평균 활용)
geo_mean = []
for i in range(len(tmp)):
    geo_mean.append((st.geometric_mean(tmp[['2021-06-10_pct','2021-06-17_pct', '2021-06-24_pct', '2021-07-01_pct', '2021-07-08_pct','2021-07-15_pct', '2021-07-22_pct', '2021-07-29_pct']].T[i] + 1) - 1))

tmp['geo_mean'] = geo_mean
tmp['gmv'] = tmp[['2021-06-03', '2021-06-10', '2021-06-17','2021-06-24', '2021-07-01', '2021-07-08', '2021-07-15', '2021-07-22','2021-07-29']].sum(axis = 1)
tmp.columns.name = ''

#### 가구/인테리어, 디지털/가전 카테고리 상승률 및 기하 평균 계산

In [50]:
tmp_2 = pd.pivot_table(data = df[(df.event_name == 'purchase_success')&((df.category1_name == '디지털/가전')|(df.category1_name == '가구/인테리어'))], index = 'category2_name', columns = 'w',values = 'user_no', aggfunc = 'nunique').reset_index()

In [53]:
# 주 단위 클릭 유저 수 상승률 계산
tmp_2['2021-06-10_pct'] = (tmp_2['2021-06-10'] - tmp_2['2021-06-03'])/tmp_2['2021-06-03']
tmp_2['2021-06-17_pct'] = (tmp_2['2021-06-17'] - tmp_2['2021-06-10'])/tmp_2['2021-06-10']
tmp_2['2021-06-24_pct'] = (tmp_2['2021-06-24'] - tmp_2['2021-06-17'])/tmp_2['2021-06-17']
tmp_2['2021-07-01_pct'] = (tmp_2['2021-07-01'] - tmp_2['2021-06-24'])/tmp_2['2021-06-24']
tmp_2['2021-07-08_pct'] = (tmp_2['2021-07-08'] - tmp_2['2021-07-01'])/tmp_2['2021-07-01']
tmp_2['2021-07-15_pct'] = (tmp_2['2021-07-15'] - tmp_2['2021-07-08'])/tmp_2['2021-07-08']
tmp_2['2021-07-22_pct'] = (tmp_2['2021-07-22'] - tmp_2['2021-07-15'])/tmp_2['2021-07-15']
tmp_2['2021-07-29_pct'] = (tmp_2['2021-07-29'] - tmp_2['2021-07-22'])/tmp_2['2021-07-22']

In [54]:
geo_mean = []

for i in range(len(tmp_2)):
    geo_mean.append((st.geometric_mean(tmp_2[['2021-06-10_pct','2021-06-17_pct', '2021-06-24_pct', '2021-07-01_pct', '2021-07-08_pct','2021-07-15_pct', '2021-07-22_pct', '2021-07-29_pct']].T[i] + 1)-1))

tmp_2['geo_mean'] = geo_mean
tmp_2['click_user_cnt'] = tmp_2[['2021-06-03', '2021-06-10', '2021-06-17','2021-06-24', '2021-07-01', '2021-07-08', '2021-07-15', '2021-07-22','2021-07-29']].sum(axis = 1)
tmp_2.columns.name = ''